## MIT

In [16]:
import yaml
import os
import sys
import pickle
import torch

sys.path.append("/root/code/iasc_imcomplete/1_MissingInstanceInfer")
from model.mit.transformer import ObjInferLayer_Type_1, ObjInferLayer_Type_2, ObjInferTransformer_type_1, ObjInferTransformer_type_2

config_path = "/root/code/iasc_imcomplete/1_MissingInstanceInfer/configs/type_1_abs4c_no_classifier_deep.yaml"
ckpt_path = "/root/code/iasc_imcomplete/checkpoints/train_mit_type_1_abs4c_no_classifier_lr_1.0e-4_deep_scheduler_0.06_0.7/epoch=299-Cls_Acc=63.405.ckpt"
with open(config_path, "r") as fp: 
    config_file = yaml.load(fp, Loader=yaml.FullLoader)



In [17]:
tf_config = config_file['mit_config']
obj_inf_type = tf_config['obj_inf_model']
no_pe = tf_config['obj_inf_no_pe']
pe_type = tf_config['obj_inf_pe_type']    

if obj_inf_type == 'type1':
        enc_layer = ObjInferLayer_Type_1(d_model=tf_config['hidden_dim'], \
            nhead=tf_config['nhead'], \
            dim_feedforward=tf_config['dim_feedforward'], \
            dropout=tf_config['dropout'], \
            activation=tf_config['tr_enc_activation'], \
            normalize_before=tf_config['pre_norm']    
            )
        transformer = ObjInferTransformer_type_1(encoder_layer=enc_layer, num_layers=tf_config['enc_layers'], no_pe=no_pe, pe_type=pe_type, \
            no_classifier=tf_config['no_classifier'], dim=tf_config['hidden_dim'], class_numb=config_file['label_nc'], contain_is_thing=tf_config['contain_is_thing'])

elif obj_inf_type == 'type2':
    enc_layer = ObjInferLayer_Type_2(d_model=tf_config['hidden_dim'], \
        nhead=tf_config['nhead'], \
        dim_feedforward=tf_config['dim_feedforward'], \
        dropout=tf_config['dropout'], \
        activation=tf_config['tr_enc_activation'], \
        normalize_before=tf_config['pre_norm']    
        )
    transformer = ObjInferTransformer_type_2(encoder_layer=enc_layer, num_layers=tf_config['enc_layers'], no_pe=no_pe, pe_type=pe_type, \
        no_classifier=tf_config['no_classifier'], dim=tf_config['hidden_dim'], class_numb=config_file['label_nc'], contain_is_thing=tf_config['contain_is_thing'])
        

In [18]:
# Load checkpoint
checkpoint = torch.load(ckpt_path)
new_state_dict = {}

for key in checkpoint['state_dict'].keys():
    if 'transformer' in key:
        new_state_dict[key.replace('transformer.', '')] = checkpoint['state_dict'][key]

transformer.load_state_dict(new_state_dict)
with open('new_mit.pkl', 'wb') as file:
    pickle.dump(transformer, file)


## MG

In [25]:
import yaml
import os
import sys
import pickle
import torch

sys.path.append("/root/code/iasc_imcomplete/2_MaskGenerator")
from model.mgan.architecture import Generator, Discriminator, MODULES


config_path = "/root/code/iasc_imcomplete/2_MaskGenerator/configs/default_no_attn_feature_condition_false.yaml"
ckpt_path = "/root/code/iasc_imcomplete/checkpoints/train_mgan_default_diff_aug_true_attn_false_extra_feature_condition_false_batch_256_0.1_0.6/epoch=17-val_fid=64.987.ckpt"
with open(config_path, "r") as fp: 
    config_file = yaml.load(fp, Loader=yaml.FullLoader)

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [26]:
mgan_config = config_file['mgan_config']
num_classes = config_file['label_nc']
module = MODULES(apply_g_sn=mgan_config['apply_g_sn'], apply_d_sn=mgan_config['apply_d_sn'])

generator = Generator(
    z_dim=mgan_config['z_dim'],
    g_shared_dim=mgan_config['g_shared_dim'],
    img_size=config_file['crop_size'],
    g_conv_dim=mgan_config['g_conv_dim'],
    apply_attn=mgan_config['apply_attn'],
    attn_g_loc=mgan_config['attn_g_loc'],
    num_classes=num_classes,
    MODULES=module,
    extra_feature_condition=mgan_config['extra_feature_condition']
    )

In [27]:
# Load checkpoint
checkpoint = torch.load(ckpt_path)
new_state_dict = {}
checkpoint['state_dict'].keys()
for key in checkpoint['state_dict'].keys():
    if 'generator' in key:
        new_state_dict[key.replace('generator.', '')] = checkpoint['state_dict'][key]

generator.load_state_dict(new_state_dict)
with open('new_isg.pkl', 'wb') as file:
    pickle.dump(generator, file)

odict_keys(['generator.linear0.bias', 'generator.linear0.weight_orig', 'generator.linear0.weight_u', 'generator.linear0.weight_v', 'generator.shared.weight', 'generator.blocks.0.0.bn1.bn.running_mean', 'generator.blocks.0.0.bn1.bn.running_var', 'generator.blocks.0.0.bn1.bn.num_batches_tracked', 'generator.blocks.0.0.bn1.gain.weight_orig', 'generator.blocks.0.0.bn1.gain.weight_u', 'generator.blocks.0.0.bn1.gain.weight_v', 'generator.blocks.0.0.bn1.bias.weight_orig', 'generator.blocks.0.0.bn1.bias.weight_u', 'generator.blocks.0.0.bn1.bias.weight_v', 'generator.blocks.0.0.bn2.bn.running_mean', 'generator.blocks.0.0.bn2.bn.running_var', 'generator.blocks.0.0.bn2.bn.num_batches_tracked', 'generator.blocks.0.0.bn2.gain.weight_orig', 'generator.blocks.0.0.bn2.gain.weight_u', 'generator.blocks.0.0.bn2.gain.weight_v', 'generator.blocks.0.0.bn2.bias.weight_orig', 'generator.blocks.0.0.bn2.bias.weight_u', 'generator.blocks.0.0.bn2.bias.weight_v', 'generator.blocks.0.0.conv2d0.bias', 'generator.bl

## BSC

In [19]:
import yaml
import os
import sys
import pickle
import torch

sys.path.append("/root/code/iasc_imcomplete/3_BkgSegmentationCompletion")
from model.bsc.architecture import BidirectionalTransformer_avgpool, BidirectionalTransformer_sconv, BidirectionalSwinTransformer_sconv
from model.bsc.my_transformer import BidirectionalTransformer_local_global, BidirectionalTransformer_local_global_mask, BidirectionalTransformer_swin_mask
from model.bsc.unet_model import UNet

config_path = "/root/code/iasc_imcomplete/3_BkgSegmentationCompletion/configs/vanilla.yaml"
ckpt_path = "/root/code/iasc_imcomplete/checkpoints/train_bsc_vanilla_0.1_0.6/epoch=186-valid_mIoU=0.602.ckpt"
with open(config_path, "r") as fp: 
    config_file = yaml.load(fp, Loader=yaml.FullLoader)

In [20]:
tf_config = config_file['tf_config']
label_nc = config_file['label_nc']
if tf_config['type'] == 'swin':
    bidi_trans = BidirectionalSwinTransformer_sconv(tf_config, label_nc=label_nc) 
elif tf_config['type'] == 'vanilla':
    bidi_trans = BidirectionalTransformer_sconv(tf_config, label_nc=label_nc) 
elif tf_config['type'] == 'unet':
    bidi_trans = UNet(tf_config, label_nc=label_nc)

Initializing Module Embedding.
Initializing Module NonDynamicallyQuantizableLinear.
Initializing Module Linear.
Initializing Module Linear.
Initializing Module NonDynamicallyQuantizableLinear.
Initializing Module Linear.
Initializing Module Linear.
Initializing Module NonDynamicallyQuantizableLinear.
Initializing Module Linear.
Initializing Module Linear.
Initializing Module NonDynamicallyQuantizableLinear.
Initializing Module Linear.
Initializing Module Linear.
Initializing Module NonDynamicallyQuantizableLinear.
Initializing Module Linear.
Initializing Module Linear.
Initializing Module NonDynamicallyQuantizableLinear.
Initializing Module Linear.
Initializing Module Linear.
Initializing Module NonDynamicallyQuantizableLinear.
Initializing Module Linear.
Initializing Module Linear.
Initializing Module NonDynamicallyQuantizableLinear.
Initializing Module Linear.
Initializing Module Linear.
Initializing Module Linear.


In [24]:
# Load checkpoint
checkpoint = torch.load(ckpt_path)
new_state_dict = {}

for key in checkpoint['state_dict'].keys():
    if 'bidi_trans' in key:
        new_state_dict[key.replace('bidi_trans.', '')] = checkpoint['state_dict'][key]

bidi_trans.load_state_dict(new_state_dict)
with open('new_bsc.pkl', 'wb') as file:
    pickle.dump(bidi_trans, file)

odict_keys(['bidi_trans.pos_emb', 'bidi_trans.final_bias', 'bidi_trans.seg2embed.weight', 'bidi_trans.ln.weight', 'bidi_trans.ln.bias', 'bidi_trans.blocks.0.MultiHeadAttention.in_proj_weight', 'bidi_trans.blocks.0.MultiHeadAttention.in_proj_bias', 'bidi_trans.blocks.0.MultiHeadAttention.out_proj.weight', 'bidi_trans.blocks.0.MultiHeadAttention.out_proj.bias', 'bidi_trans.blocks.0.LayerNorm1.weight', 'bidi_trans.blocks.0.LayerNorm1.bias', 'bidi_trans.blocks.0.LayerNorm2.weight', 'bidi_trans.blocks.0.LayerNorm2.bias', 'bidi_trans.blocks.0.MLP.0.weight', 'bidi_trans.blocks.0.MLP.0.bias', 'bidi_trans.blocks.0.MLP.2.weight', 'bidi_trans.blocks.0.MLP.2.bias', 'bidi_trans.blocks.1.MultiHeadAttention.in_proj_weight', 'bidi_trans.blocks.1.MultiHeadAttention.in_proj_bias', 'bidi_trans.blocks.1.MultiHeadAttention.out_proj.weight', 'bidi_trans.blocks.1.MultiHeadAttention.out_proj.bias', 'bidi_trans.blocks.1.LayerNorm1.weight', 'bidi_trans.blocks.1.LayerNorm1.bias', 'bidi_trans.blocks.1.LayerNorm2.